In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 4

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/additive/combosciplex/adata_ood_with_predictions_{split}.h5ad")

In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_train
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].X.toarray()
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8678234020868937,
 'decoded_ood_sinkhorn_div_1': 428.4073130289714,
 'decoded_ood_sinkhorn_div_10': 396.7443033854167,
 'decoded_ood_sinkhorn_div_100': 112.9093526204427,
 'decoded_ood_e_distance': 23.468859365447713,
 'decoded_ood_mmd': 0.07505907863378525}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9905793690681457,
 'decoded_test_e_distance': 1.7493059700489777,
 'decoded_test_mmd_distance': 0.009574546851217747}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8678234020868937,
 'decoded_ood_sinkhorn_div_1': 428.4073130289714,
 'decoded_ood_sinkhorn_div_10': 396.7443033854167,
 'decoded_ood_sinkhorn_div_100': 112.9093526204427,
 'decoded_ood_e_distance': 23.468859365447713,
 'decoded_ood_mmd': 0.07505907863378525}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -3.561306277910868,
 'encoded_ood_sinkhorn_div_1': 22.012921651204426,
 'encoded_ood_sinkhorn_div_10': 14.328825950622559,
 'encoded_ood_sinkhorn_div_100': 8.085921287536621,
 'encoded_ood_e_distance': 14.226102918254128,
 'encoded_ood_mmd': 0.05197146348655224}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8495323260625204,
 'deg_ood_sinkhorn_div_1': 36.72677596410116,
 'deg_ood_sinkhorn_div_10': 17.336824417114258,
 'deg_ood_sinkhorn_div_100': 4.93192704518636,
 'deg_ood_e_distance': 8.879143970137717,
 'deg_ood_mmd': 0.049823139794170856}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.9957353043556213,
 'deg_test_sinkhorn_div_1': 12.208970432281495,
 'deg_test_sinkhorn_div_10': 0.8475563049316406,
 'deg_test_sinkhorn_div_100': 0.14981258392333985,
 'deg_test_e_distance': 0.25317418185254614,
 'deg_test_mmd': 0.007509394511580467}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9905793690681457,
 'decoded_test_e_distance': 1.7493059700489777,
 'decoded_test_mmd_distance': 0.009574546851217747}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8678234020868937,
 'decoded_ood_sinkhorn_div_1': 428.4073130289714,
 'decoded_ood_sinkhorn_div_10': 396.7443033854167,
 'decoded_ood_sinkhorn_div_100': 112.9093526204427,
 'decoded_ood_e_distance': 23.468859365447713,
 'decoded_ood_mmd': 0.07505907863378525}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8495323260625204,
 'deg_ood_sinkhorn_div_1': 36.72677596410116,
 'deg_ood_sinkhorn_div_10': 17.336824417114258,
 'deg_ood_sinkhorn_div_100': 4.93192704518636,
 'deg_ood_e_distance': 8.879143970137717,
 'deg_ood_mmd': 0.049823139794170856}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9905793690681457,
 'decoded_test_e_distance': 1.7493059700489777,
 'decoded_test_mmd_distance': 0.009574546851217747}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.9925168390446301,
 'encoded_test_sinkhorn_div_1': 1.9724458503723143,
 'encoded_test_sinkhorn_div_10': 0.3079781723022461,
 'encoded_test_sinkhorn_div_100': 0.10843212127685548,
 'encoded_test_e_distance': 0.19064514453919792,
 'encoded_test_mmd': 0.007178473733365536}

In [25]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': -1.3508422374725342,
  'sinkhorn_div_1': 27.850257873535156,
  'sinkhorn_div_10': 19.074356079101562,
  'sinkhorn_div_100': 8.750804901123047,
  'e_distance': 14.268834382312775,
  'mmd': 0.05930804},
 'Givinostat+Crizotinib': {'r_squared': -7.093914985656738,
  'sinkhorn_div_1': 20.947845458984375,
  'sinkhorn_div_10': 12.781112670898438,
  'sinkhorn_div_100': 6.503089904785156,
  'e_distance': 11.095320260100209,
  'mmd': 0.04614583},
 'Givinostat+SRT2104': {'r_squared': -4.896740436553955,
  'sinkhorn_div_1': 19.250577926635742,
  'sinkhorn_div_10': 11.047801971435547,
  'sinkhorn_div_100': 4.548137664794922,
  'e_distance': 7.110489041526762,
  'mmd': 0.039788954},
 'Givinostat+Sorafenib': {'r_squared': -3.5398497581481934,
  'sinkhorn_div_1': 42.846641540527344,
  'sinkhorn_div_10': 33.30554962158203,
  'sinkhorn_div_100': 23.891937255859375,
  'e_distance': 44.71748616403306,
  'mmd': 0.12467668},
 'control+Givinostat': {'r_squared': -5.38934

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/additive"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
